In [1]:
## Import packages
## Import packages
import os
import mne
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from scipy.signal import butter, sosfiltfilt, sosfreqz  
from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_curve, auc

In [2]:
## Define directories
data_dir = '/home/inffzy/Desktop/cogs189/cogs189_final_project/data'

In [3]:
## Raw data directory names
bc4_2a_processed_name = 'bci_competition_4_2a_processed'
bc3_3a_processed_name = 'bci_competition_3_3a_processed'

## Create list of all data paths
bc4_2a_processed_data_paths = []
bc3_3a_processed_data_paths = []

## Add bc4_2a data paths
for subject_idx in range(1, 10):
    subject_name = 'A0' + str(subject_idx) + 'T'
    bc4_2a_processed_data_paths.append(
        os.path.join(data_dir, bc4_2a_processed_name, subject_name + '.npz'))
    
## Add bc3_3a data paths
for subject_idx in range(1, 4):
    subject_name = 'bc3_3a_s' + str(subject_idx)
    bc3_3a_processed_data_paths.append(
        os.path.join(data_dir, bc3_3a_processed_name, subject_name + '.npz'))

In [4]:
bc3_3a_s1_data_npz = np.load(bc3_3a_processed_data_paths[0])

bc3_3a_s1_data = bc3_3a_s1_data_npz['processed_motor_imagery_data']
bc3_3a_s1_labels = bc3_3a_s1_data_npz['descriptions']

In [5]:
bc3_3a_s1_labels

array([771, 772, 771, 771, 771, 770, 772, 772, 769, 770, 770, 769, 770,
       772, 769, 770, 771, 769, 769, 771, 769, 771, 769, 769, 769, 770,
       772, 771, 770, 772, 770, 770, 771, 772, 769, 770, 772, 771, 769,
       772, 769, 772, 771, 769, 769, 770, 769, 772, 772, 770, 770, 771,
       772, 769, 772, 772, 771, 769, 771, 772, 771, 771, 771, 772, 770,
       772, 772, 769, 769, 771, 770, 772, 770, 769, 772, 770, 770, 772,
       769, 770, 770, 772, 770, 771, 772, 772, 771, 771, 772, 770, 771,
       770, 769, 771, 772, 769, 770, 769, 769, 771, 772, 771, 771, 769,
       771, 772, 770, 770, 770, 772, 772, 771, 771, 772, 769, 771, 769,
       771, 771, 769, 770, 770, 769, 769, 770, 770, 771, 772, 771, 769,
       769, 770, 770, 769, 771, 771, 772, 770, 771, 769, 769, 770, 772,
       770, 772, 772, 772, 770, 771])

In [6]:
bc4_2a_s1_data_npz = np.load(bc4_2a_processed_data_paths[0])

bc4_2a_s1_data = bc4_2a_s1_data_npz['processed_motor_imagery_data']
bc4_2a_s1_labels = bc4_2a_s1_data_npz['descriptions']

In [7]:
bc4_2a_s1_labels

array([772, 771, 770, 769, 769, 770, 771, 772, 770, 771, 769, 769, 769,
       772, 770, 770, 769, 769, 771, 769, 770, 772, 771, 769, 772, 772,
       770, 772, 772, 770, 769, 770, 771, 771, 771, 772, 771, 769, 772,
       770, 771, 770, 771, 772, 771, 769, 769, 769, 772, 770, 771, 769,
       771, 772, 769, 771, 771, 769, 771, 770, 772, 772, 772, 769, 772,
       770, 772, 770, 769, 771, 770, 769, 771, 771, 769, 771, 772, 772,
       770, 769, 770, 772, 770, 772, 771, 770, 770, 770, 771, 769, 770,
       771, 771, 772, 769, 769, 770, 772, 772, 772, 770, 769, 771, 770,
       772, 769, 772, 771, 770, 772, 772, 769, 770, 770, 771, 772, 770,
       769, 769, 772, 770, 769, 771, 770, 770, 771, 769, 772, 771, 771,
       771, 771, 769, 770, 769, 770, 769, 769, 771, 771, 770, 771, 772,
       769, 772, 769, 769, 770, 772, 771, 770, 772, 771, 772, 771, 772,
       770, 770, 772, 769, 770, 770, 770, 771, 772, 769, 772, 769, 771,
       769, 772, 769, 771, 769, 770, 771, 771, 772, 769, 770, 77

In [9]:
num_bc4_2a_channels = bc4_2a_s1_data.shape[1]

In [10]:
bc4_2a_s1_data.shape

(273, 25, 750)

In [11]:
## Separate subject data by trial label
bc4_2a_s1_data_c1 = bc4_2a_s1_data[bc4_2a_s1_labels == 769]
bc4_2a_s1_data_c2 = bc4_2a_s1_data[bc4_2a_s1_labels == 770]
bc4_2a_s1_data_c3 = bc4_2a_s1_data[bc4_2a_s1_labels == 771]
bc4_2a_s1_data_c4 = bc4_2a_s1_data[bc4_2a_s1_labels == 772]

num_bc4_2a_s1_c1_trials = bc4_2a_s1_data_c1.shape[0]
num_bc4_2a_s1_c2_trials = bc4_2a_s1_data_c2.shape[0]
num_bc4_2a_s1_c3_trials = bc4_2a_s1_data_c3.shape[0]
num_bc4_2a_s1_c4_trials = bc4_2a_s1_data_c4.shape[0]

In [13]:
## Calculate normalized spatial covariance of each trial
bc4_2a_s1_data_c1_trial_covs = np.zeros((num_bc4_2a_s1_c1_trials, num_bc4_2a_channels, num_bc4_2a_channels))
bc4_2a_s1_data_c2_trial_covs = np.zeros((num_bc4_2a_s1_c2_trials, num_bc4_2a_channels, num_bc4_2a_channels))
bc4_2a_s1_data_c3_trial_covs = np.zeros((num_bc4_2a_s1_c3_trials, num_bc4_2a_channels, num_bc4_2a_channels))
bc4_2a_s1_data_c4_trial_covs = np.zeros((num_bc4_2a_s1_c4_trials, num_bc4_2a_channels, num_bc4_2a_channels))

for i in range(num_bc4_2a_s1_c1_trials):
    bc4_2a_s1_data_c1_trial = bc4_2a_s1_data_c1[i]
    
    ## Normalize spatial covariance
    bc4_2a_s1_data_c1_trial_prod = bc4_2a_s1_data_c1_trial @ bc4_2a_s1_data_c1_trial.T
    bc4_2a_s1_data_c1_trial_cov = bc4_2a_s1_data_c1_trial_prod / np.trace(bc4_2a_s1_data_c1_trial_prod)
    
    bc4_2a_s1_data_c1_trial_covs[i] = bc4_2a_s1_data_c1_trial_cov

for i in range(num_bc4_2a_s1_c2_trials):
    bc4_2a_s1_data_c2_trial = bc4_2a_s1_data_c2[i]
    
    ## Normalize spatial covariance
    bc4_2a_s1_data_c2_trial_prod = bc4_2a_s1_data_c2_trial @ bc4_2a_s1_data_c2_trial.T
    bc4_2a_s1_data_c2_trial_cov = bc4_2a_s1_data_c2_trial_prod / np.trace(bc4_2a_s1_data_c2_trial_prod)
    bc4_2a_s1_data_c2_trial_covs[i] = bc4_2a_s1_data_c2_trial_cov

for i in range(num_bc4_2a_s1_c3_trials):
    bc4_2a_s1_data_c3_trial = bc4_2a_s1_data_c3[i]
    
    ## Normalize spatial covariance
    bc4_2a_s1_data_c3_trial_prod = bc4_2a_s1_data_c3_trial @ bc4_2a_s1_data_c3_trial.T
    bc4_2a_s1_data_c3_trial_cov = bc4_2a_s1_data_c3_trial_prod / np.trace(bc4_2a_s1_data_c3_trial_prod)
    bc4_2a_s1_data_c3_trial_covs[i] = bc4_2a_s1_data_c3_trial_cov

for i in range(num_bc4_2a_s1_c4_trials):
    bc4_2a_s1_data_c4_trial = bc4_2a_s1_data_c4[i]
    
    ## Normalize spatial covariance
    bc4_2a_s1_data_c4_trial_prod = bc4_2a_s1_data_c4_trial @ bc4_2a_s1_data_c4_trial.T
    bc4_2a_s1_data_c4_trial_cov = bc4_2a_s1_data_c4_trial_prod / np.trace(bc4_2a_s1_data_c4_trial_prod)
    bc4_2a_s1_data_c4_trial_covs[i] = bc4_2a_s1_data_c4_trial_cov

In [14]:
## Calculate averaged normalized spatial covariance
bc4_2a_s1_data_c1_trial_cov_avg = np.mean(bc4_2a_s1_data_c1_trial_covs, axis=0)
bc4_2a_s1_data_c2_trial_cov_avg = np.mean(bc4_2a_s1_data_c2_trial_covs, axis=0)
bc4_2a_s1_data_c3_trial_cov_avg = np.mean(bc4_2a_s1_data_c3_trial_covs, axis=0)
bc4_2a_s1_data_c4_trial_cov_avg = np.mean(bc4_2a_s1_data_c4_trial_covs, axis=0)

In [15]:
## Calculate composite spatial covariance
R12 = bc4_2a_s1_data_c1_trial_cov_avg + bc4_2a_s1_data_c2_trial_cov_avg
R13 = bc4_2a_s1_data_c1_trial_cov_avg + bc4_2a_s1_data_c3_trial_cov_avg
R14 = bc4_2a_s1_data_c1_trial_cov_avg + bc4_2a_s1_data_c4_trial_cov_avg
R23 = bc4_2a_s1_data_c2_trial_cov_avg + bc4_2a_s1_data_c3_trial_cov_avg
R24 = bc4_2a_s1_data_c2_trial_cov_avg + bc4_2a_s1_data_c4_trial_cov_avg
R34 = bc4_2a_s1_data_c3_trial_cov_avg + bc4_2a_s1_data_c4_trial_cov_avg

In [16]:
## Eigen-decompose composite spatial covariance
R12_eigval, R12_eigvec = np.linalg.eig(R12)
R13_eigval, R13_eigvec = np.linalg.eig(R13)
R14_eigval, R14_eigvec = np.linalg.eig(R14)
R23_eigval, R23_eigvec = np.linalg.eig(R23)
R24_eigval, R24_eigvec = np.linalg.eig(R24)
R34_eigval, R34_eigvec = np.linalg.eig(R34)

In [17]:
## Create diagonal matrix of eigenvalues
R12_eigval_diag = np.zeros_like(R12_eigvec)
R13_eigval_diag = np.zeros_like(R13_eigvec)
R14_eigval_diag = np.zeros_like(R14_eigvec)
R23_eigval_diag = np.zeros_like(R23_eigvec)
R24_eigval_diag = np.zeros_like(R24_eigvec)
R34_eigval_diag = np.zeros_like(R34_eigvec)

for i in range(R12_eigval.shape[0]):
    R12_eigval_diag[i, i] = R12_eigval[i]
    R13_eigval_diag[i, i] = R13_eigval[i]
    R14_eigval_diag[i, i] = R14_eigval[i]
    R23_eigval_diag[i, i] = R23_eigval[i]
    R24_eigval_diag[i, i] = R24_eigval[i]
    R34_eigval_diag[i, i] = R34_eigval[i]

In [18]:
## Calculate Whitening transformation matrix
P12 = np.linalg.inv(np.sqrt(R12_eigval_diag)) @ R12_eigvec.T
P13 = np.linalg.inv(np.sqrt(R13_eigval_diag)) @ R13_eigvec.T
P14 = np.linalg.inv(np.sqrt(R14_eigval_diag)) @ R14_eigvec.T
P23 = np.linalg.inv(np.sqrt(R23_eigval_diag)) @ R23_eigvec.T
P24 = np.linalg.inv(np.sqrt(R24_eigval_diag)) @ R24_eigvec.T
P34 = np.linalg.inv(np.sqrt(R34_eigval_diag)) @ R34_eigvec.T

In [19]:
## Whitening Transform average covariance
S12_1 = P12 @ bc4_2a_s1_data_c1_trial_cov_avg @ P12.T
S12_2 = P12 @ bc4_2a_s1_data_c2_trial_cov_avg @ P12.T
S13_1 = P13 @ bc4_2a_s1_data_c1_trial_cov_avg @ P13.T
S13_3 = P13 @ bc4_2a_s1_data_c3_trial_cov_avg @ P13.T
S14_1 = P14 @ bc4_2a_s1_data_c1_trial_cov_avg @ P14.T
S14_4 = P14 @ bc4_2a_s1_data_c4_trial_cov_avg @ P14.T
S23_2 = P23 @ bc4_2a_s1_data_c2_trial_cov_avg @ P23.T
S23_3 = P23 @ bc4_2a_s1_data_c3_trial_cov_avg @ P23.T
S24_2 = P24 @ bc4_2a_s1_data_c2_trial_cov_avg @ P24.T
S24_4 = P24 @ bc4_2a_s1_data_c4_trial_cov_avg @ P24.T
S34_3 = P34 @ bc4_2a_s1_data_c3_trial_cov_avg @ P34.T
S34_4 = P34 @ bc4_2a_s1_data_c4_trial_cov_avg @ P34.T

In [20]:
## Eigen-decompose of whitening transformed average covariance
S12_1_eigval, S12_1_eigvec = np.linalg.eig(S12_1)
S12_2_eigval, S12_2_eigvec = np.linalg.eig(S12_2)
S13_1_eigval, S13_1_eigvec = np.linalg.eig(S13_1)
S13_3_eigval, S13_3_eigvec = np.linalg.eig(S13_3)
S14_1_eigval, S14_1_eigvec = np.linalg.eig(S14_1)
S14_4_eigval, S14_4_eigvec = np.linalg.eig(S14_4)
S23_2_eigval, S23_2_eigvec = np.linalg.eig(S23_2)
S23_3_eigval, S23_3_eigvec = np.linalg.eig(S23_3)
S24_2_eigval, S24_2_eigvec = np.linalg.eig(S24_2)
S24_4_eigval, S24_4_eigvec = np.linalg.eig(S24_4)
S34_3_eigval, S34_3_eigvec = np.linalg.eig(S34_3)
S34_4_eigval, S34_4_eigvec = np.linalg.eig(S34_4)

In [36]:
S12_1_eigval + S12_2_eigval

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [33]:
## Take the largest 3 and smallest 3 eigenvectors to contruct projection matrix W
num_dim_per_label = 3
top_n_indices, bot_n_indices = get_top_bot_n_indices(S12_1_eigval, num_dim_per_label)

S12_1_eigvec_extracted = S12_1_eigvec[top_n_indices + bot_n_indices]

W12 = S12_1_eigvec_extracted @ P12

In [34]:
W12.shape

(6, 25)

In [29]:
def get_top_bot_n_indices(arr_np, n):
    sort_indices = np.argsort(arr_np)
    top_n_indices = sort_indices[:n]
    bot_n_indices = sort_indices[-n:]
    return list(top_n_indices), list(bot_n_indices)

In [35]:
bc4_2a_s1_data_c1.shape

(69, 25, 750)

In [27]:
top_n_indices

array([1, 6, 7])

In [28]:
bot_n_indices

array([3, 2, 0])

In [ ]:
R12_eigval_diag.shape

In [ ]:
eigvec_12.shape

In [ ]:
P12.shape

In [ ]:
S1.shape